# Guía 1: Operaciones matemáticas básicas

## Aspectos teóricos

1. Deduzca la fórmula de 5 puntos para la derivación numérica.
2. Deduzca la generalización de la regla de Simpson.
3. Dada la integral $$\int_1^b x^{-2} g(x) dx$$ donde $g(x)$ es una función que tiende a un valor constante para grandes valores de $x$, realice un cambio de variables que la transforme en una expresión más adecuada para su evolución numérica, cuando $b$ toma valores muy grandes.
4. Escriba las ecuaciones utilizadas en el método de Newton-Raphson y el de la Secante.
5. Suponga que usted desea ajustar la función $f(x; a_1, a_2,..., a_k)$ con parámetros $a_1, a_2, ..., a_k$ a partir de un conjunto de puntos $\lbrace x_i, y_i \rbrace$ con $i=1, ..., N$. Escriba formalmente la función a minimizar en base al método de los cuadrados mínimos.

## Ejercicios de programación

1. Escriba una función en Python para encontrar numéricamente las raíces de $f(x) = x^2 - 5$. Compare el número de iteraciones necesarias para lograr la convergencia con distintos valores de tolerancia. Compare para los distintos métodos de búsqueda: Bisección, Newton-Raphson y Secante.

2. Escriba una función en Python para evaluar la derivada de la función $\sin(x)$ en el intervalo $[2\pi, 4\pi]$. Analice el resultado para diferentes valores del incremento finito $h$. Estudie cómo varía el error numérico total en función de $h$ y de la precisión utilizada. Analice el error local en cada zona del dominio.

3. Escriba una función en Python para evaluar la integral definida de la función $\cos(x)$ en el intervalo $[0, 1]$. Estudie el efecto del número de particiones del intervalo sobre el error numérico.

4. Use la función `simplex` proporcionada para encontrar la distancia interatómica de equilibrio de la molécula NaCl a partir del siguiente potencial de interacción entre los iones
$$
V(r) = - \frac{e^2}{r} + \alpha e^{- \frac{r}{\rho}}
$$
donde $e^2 = 14.4 eV \cdot A$, $\alpha = 1.09 \cdot 10^3 eV$ y $\rho = 0.33 A$.

5. Usar la funció `simplex` proporcionada para ajustar la función $f(x, a_1, a_2) = e^{-a_1 x} + e^{-a_2 x}$ al conjunto de datos que se encuentra en el archivo `datos.txt`. Tenga en cuenta el ejercicio 5 de la parte teórica. (Sugerencia: Buscar los valores $a_1$ y $a_2$ en el intervalor $[0,1]$).